In [1]:
from pydantic import BaseModel
from typing import Annotated, List, TypedDict
from langgraph.graph import StateGraph, START,END
from langchain_groq import ChatGroq
from langgraph.graph.message import add_messages
from langchain_core.messages import AnyMessage, HumanMessage, RemoveMessage, AIMessage

from langgraph.prebuilt import ToolNode
from langgraph.prebuilt import tools_condition

from langgraph.checkpoint.memory import MemorySaver

from IPython.display import Image, display

In [2]:
class State(TypedDict):
    messages:Annotated[List[AnyMessage],add_messages]
    summary:str

In [3]:
model = ChatGroq(model="llama-3.1-8b-instant",temperature=0)

In [4]:
def assistant(state:State):
    summary = state.get("summary")
    
    if summary:
        summary_prompt = HumanMessage(f"This is the summary of previous conversations: {summary}")
        return {"messages" : model.invoke([summary_prompt]+state['messages'])}
    else:
        return {"messages":model.invoke(state['messages'])}

In [ ]:
def summary_func(state:State):
    summary = state.get("summary")
    
    if summary:
        summary_prompt = HumanMessage(f"This is the summary from previous chat: {summary}\nExtend this for following messages: ")
        summary = model.invoke([summary_prompt]+state['messages']).content
    else:
        summary_prompt = HumanMessage("Summarize the above messages in less than 300 words: ")
        summary = model.invoke(state['messages']+[summary_prompt]).content
    
    delete_messages = [RemoveMessage(id = m.id) for m in state["messages"][:-2]]
    return {"messages":delete_messages,"summary":summary}

In [ ]:
def summarize_condition(state:State):
    if len(state['messages'])>6:
        return "summary_func"
    
    return END

In [ ]:
memory = MemorySaver()

In [ ]:
builder = StateGraph(State)
builder.add_node("assistant",assistant)
builder.add_node("summary_func",summary_func)

builder.add_edge(START,"assistant")
builder.add_conditional_edges("assistant",summarize_condition,{"summary_func":"summary_func",END:END})
builder.add_edge("summary_func",END)

graph = builder.compile(checkpointer=memory)
display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
result = graph.invoke({"messages":"how are you"},config = {"configurable":{"thread_id":"firstchat"}})
result

In [ ]:
result = graph.invoke({"messages":"explain me about gravity"},config = {"configurable":{"thread_id":"firstchat"}})
result

In [ ]:
result = graph.invoke({"messages":"how does it effect planets"},config = {"configurable":{"thread_id":"firstchat"}})
result

In [ ]:
result = graph.invoke({"messages":"who discovered it"},config = {"configurable":{"thread_id":"firstchat"}})
result

In [ ]:
config = {"configurable":{"thread_id":"firstchat"}}
graph.get_state(config).values.get("summary","")

In [ ]:
d = {"a":9}
print(type(d.get('b')))
if d.get("b"):
    print("found b")
else:
    print("not found b")

In [ ]:
d = {"a":9}
b = d.get("b")
if b:
    print("found b")
else:
    print("not found b")